In [ ]:
cd /content/drive/MyDrive

In [ ]:
!unzip Challenge_images.zip

In [ ]:
import os

In [ ]:
data_dir = "./Challenge_images"

In [ ]:
os.mkdir(os.path.join(data_dir,"single_class"))

In [ ]:
os.listdir(os.path.join(data_dir,"single_class"))

In [ ]:
files_list = os.listdir(data_dir)
files_list.remove("single_class")

In [ ]:
import shutil

In [ ]:
for single_img_file in files_list:
    
    shutil.move(src=os.path.join(data_dir,single_img_file),dst=os.path.join(data_dir,"single_class"))

In [ ]:
os.listdir(data_dir)

In [ ]:
os.listdir(os.path.join(data_dir,"single_class"))

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [ ]:
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(416),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
image_datasets = datasets.ImageFolder(data_dir,data_transforms)
dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=4,shuffle=True, num_workers=4)
dataset_size = len(image_datasets)
class_names = image_datasets.classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
from torchvision.models import resnet18
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

In [ ]:
train_nodes,eval_nodes = get_graph_node_names(resnet18())

In [ ]:
eval_nodes

In [ ]:
inputs,classes = next(iter(dataloaders))

In [ ]:
inputs.shape

In [ ]:
! python setup.py install

In [ ]:
torch.ops.load_library("/content/drive/MyDrive/build/lib.linux-x86_64-3.7/repeatInterleave.cpython-37m-x86_64-linux-gnu.so")
torch.ops.load_library("/content/drive/MyDrive/build/lib.linux-x86_64-3.7/reduction.cpython-37m-x86_64-linux-gnu.so")

In [ ]:
from torch.onnx import register_custom_op_symbolic

In [ ]:
def symbolic_repeatInterleave(g,tnsr):

  return g.op("custom_domain_RepInterleave::onnx_repeatInterleave",tnsr)

In [ ]:
register_custom_op_symbolic("RepInterleave::repeatInterleave",symbolic_repeatInterleave,9)

In [ ]:
def symbolic_reduction(g,tnsr1,tnsr2,tnsr3,tnsr4):

  return g.op("custom_domain_Reduce::onnx_reduction",tnsr1,tnsr2,tnsr3,tnsr4)

In [ ]:
register_custom_op_symbolic("Reduce::reduction",symbolic_reduction,9)

In [ ]:
class ReductionResNet(nn.Module):
    
    def __init__(self):
        
        super(ReductionResNet,self).__init__()
        
        pretrained_resnet18_model = resnet18(pretrained=True)
        
        self.pretrained_resnet18_head = create_feature_extractor(model=pretrained_resnet18_model,
                                                               return_nodes={'layer1.1.relu_1':'layer1',
                                                                            'layer2.1.relu_1':'layer2',
                                                                            'layer3.1.relu_1':'layer3',
                                                                            'layer4.1.relu_1':'layer4'})
        
    def forward(self,x):
        
        x= self.pretrained_resnet18_head(x)

        two_dim_pooling = nn.AdaptiveAvgPool2d(output_size=(1,1))

        layer1_output = two_dim_pooling(x['layer1']).view(1,x['layer1'].shape[1])
        layer2_output = two_dim_pooling(x['layer2']).view(1,x['layer2'].shape[1])
        layer3_output = two_dim_pooling(x['layer3']).view(1,x['layer3'].shape[1])
        layer4_output = two_dim_pooling(x['layer4']).view(1,x['layer4'].shape[1])

        layer1_output = torch.ops.RepInterleave.repeatInterleave(layer1_output).reshape(1,512)
        layer2_output = torch.ops.RepInterleave.repeatInterleave(layer2_output).reshape(1,512)
        layer3_output = torch.ops.RepInterleave.repeatInterleave(layer3_output).reshape(1,512)
        layer4_output = torch.ops.RepInterleave.repeatInterleave(layer4_output).reshape(1,512)

        image_embedding = torch.ops.Reduce.reduction(layer1_output,layer2_output,layer3_output,layer4_output)

        return image_embedding

In [ ]:
our_model = ReductionResNet()

In [ ]:
print(our_model)

In [ ]:
input_tensor = torch.randn(1,3,416,416)
model_output = our_model(input_tensor)

In [ ]:
model_output.shape

In [ ]:
! pip install onnxruntime